In [3]:
import sys
print(sys.executable)

from lawson_quant_library.util import Calendar
from lawson_quant_library.parameter import IRCurve, DivCurve, EQVol
from lawson_quant_library.instrument import EQOption

REFERENCE_DATE = "2026-01-06"
cal = Calendar("US:NYSE")
spot = 100.0

/Users/lawsonprendergast/miniforge3/envs/lawson-quant/bin/python


In [4]:
ir_curve = IRCurve.from_deposit_quotes(
    {"1M": 0.0500, "3M": 0.0520, "6M": 0.0530, "1Y": 0.0550},
    reference_date=REFERENCE_DATE,
    name="USD_IR",
)

div_curve = DivCurve(0.00)

In [5]:
tenors = ["1M", "3M", "6M", "1Y"]
strikes = [80, 90, 100, 110, 120]

# vols[tenor_index][strike_index]
vol_grid = [
    [0.28, 0.24, 0.20, 0.21, 0.23],  # 1M
    [0.26, 0.22, 0.19, 0.20, 0.22],  # 3M
    [0.25, 0.21, 0.18, 0.19, 0.21],  # 6M
    [0.24, 0.20, 0.17, 0.18, 0.20],  # 1Y
]

vol = EQVol(currency="USD")
vol.set_surface_vol(
    strikes=strikes,
    tenors=tenors,
    vols=vol_grid,
    reference_date=REFERENCE_DATE,
)

In [6]:
prices = {}
for t in tenors:
    maturity = cal.add_tenor(REFERENCE_DATE, t)
    prices[t] = {}
    for k in strikes:
        opt = EQOption(
            spot=spot,
            strike=float(k),
            maturity_date=maturity,
            option_type="call",
            ir_curve=ir_curve,
            div_curve=div_curve,
            vol=vol,
            pricing_engine="bs_analytic",
        )
        prices[t][k] = opt.price()

prices

{'1M': {80: 20.344061636814953,
  90: 10.54676728320051,
  100: 2.537574195318202,
  110: 0.19279588414611862,
  120: 0.008933988968604032},
 '3M': {80: 21.164747451057345,
  90: 11.872247463940335,
  100: 4.406031317421481,
  110: 1.1764007536359113,
  120: 0.3089280199203187},
 '6M': {80: 22.597561367152956,
  90: 13.723213254508327,
  100: 6.372229002044891,
  110: 2.6582524848978952,
  120: 1.1903974715023746},
 '1Y': {80: 25.45321986795321,
  90: 16.925743560003717,
  100: 9.526075964610488,
  110: 5.386235569178359,
  120: 3.3378861432671827}}

In [7]:
greeks = {}
for t in tenors:
    maturity = cal.add_tenor(REFERENCE_DATE, t)
    greeks[t] = {}
    for k in strikes:
        opt = EQOption(
            spot=spot,
            strike=float(k),
            maturity_date=maturity,
            option_type="call",
            ir_curve=ir_curve,
            div_curve=div_curve,
            vol=vol,
            pricing_engine="bs_analytic",
        )
        greeks[t][k] = {"delta": opt.delta(), "vega": opt.vega()}

# Example: ATM greeks at 3M
greeks["3M"][100.0]

{'delta': 0.5722179651480831, 'vega': 19.48454299383948}

In [8]:
def call(k, tenor):
    return EQOption(
        spot=spot,
        strike=float(k),
        maturity_date=cal.add_tenor(REFERENCE_DATE, tenor),
        option_type="call",
        ir_curve=ir_curve,
        div_curve=div_curve,
        vol=vol,
        pricing_engine="bs_analytic",
    )

def put(k, tenor):
    return EQOption(
        spot=spot,
        strike=float(k),
        maturity_date=cal.add_tenor(REFERENCE_DATE, tenor),
        option_type="put",
        ir_curve=ir_curve,
        div_curve=div_curve,
        vol=vol,
        pricing_engine="bs_analytic",
    )

tenor = "3M"
atm = 100.0

straddle_price = call(atm, tenor).price() + put(atm, tenor).price()
rr_proxy_price = call(110.0, tenor).price() - put(90.0, tenor).price()

straddle_price, rr_proxy_price

(7.5475961273242325, 0.44217314646243466)

In [9]:
tenor = "6M"
strike_sweep = [70, 80, 90, 100, 110, 120, 130]

sweep_prices = []
for k in strike_sweep:
    sweep_prices.append(call(k, tenor).price())

list(zip(strike_sweep, sweep_prices))

[(70, 31.968873139989075),
 (80, 22.597561367152956),
 (90, 13.723213254508327),
 (100, 6.372229002044891),
 (110, 2.6582524848978952),
 (120, 1.1903974715023746),
 (130, 0.5649424570557939)]